## QHDOPT for nonlinear programming

In this notebook, we demonstrate how to employ QHDOPT to solve a nonlinear programming problem with box constraints. 

Our target problem is $$\min \ f(x)= y^2z - xz + x^2y,$$ where $x,y,z$ are continuous variables ranging from 0 to 1.

We employ the SymPy mode of QHDOPT to input this problem.

### 1. Create problem instance

First, we import the class QHD from our package QHDOPT, implying the solver's algorithm.

In [1]:
from qhdopt import QHD

Next, we import SymPy library to enable symbolic expression of objective functions.

In [2]:
from sympy import symbols

x, y, z = symbols('x y z')
f = (y**2) * z - x * z + (x**2) * y

Now, we create a problem instance, stored in a variable `model`. The `SymPy` input format requires both the objective function `f` and a list of symbols (e.g., `[x, y, z]`) to construct the problem. By default, it sets the constraints to the unit box $[0,1]^n$, where $n$ is the number of continuous variables. You may override the bounds by `bounds=(l,r)` or `bounds=[(l1, r1), ..., (ln, rn)]`.

In [3]:
model = QHD.SymPy(f, [x, y, z])

### 2. Solve with D-Wave

Then, we illustrate how to solve the problem with QHDOPT's solvers. We consider the D-Wave solver first. 

We can configure the D-Wave solver by running `model.dwave_setup` with all the parameters set. The mandatory parameter is the resolution $r$, which we set as 8. The API key can be either directly input by setting `api_key` or from a file. 

You may also set the annealing schedule, chain strength, embedding schemes, etc. Here we use default parameters. 

In [5]:
model.dwave_setup(8, api_key_from_file='dwave_api_key.txt', post_processing_method='TNC')

To compile, send the job to run, and post-processing, you can run `model.optimize`. Setting `verbose=1` outputs more runtime information.

In [6]:
response = model.optimize(verbose = 1)

Backend QPU Time: 0.02633476
Overhead Time: 2.628731968591919

* Runtime breakdown
SimuQ compilation: 0.001 s
Backend runtime: 2.655 s
Decoding time: 0.002 s
Classical (Fine-tuning) time: 0.267 s
* Total time: 2.925 s

* Coarse solution
Minimizer: [1. 0. 1.]
Minimum: -1.0

* Fine-tuned solution
Minimizer: [1. 0. 1.]
Minimum: -1.0



Here, the coarse solution is one of the decoded solutions directly from D-Wave devices, fined-tuned solution is the best solution obtained by using classical local solvers to refine the coarse solutions.

The D-Wave solver returns a global minimum at $x=[1,0,1]^T$, with the minimum value $-1.0$. After fine-tuning, the minimum does not change in this case. 

A runtime breakdown is also provided to exhibit the time consumption of each step in the solver.

The Response object holds all relevant solution information in a structured way. It also contains more debugging and time information.

### 3. Solve with QuTiP

The QHD algorithm can be deployed to different backends, thanks to SimuQ's hamiltonian-based compilation scheme. Here we demonstrate how to use a QuTiP-based solver to implement QHD and solve the nonlinear programming problem.

The workflow follows the same style. we first setup the QuTiP solver, then solve with `optimize`. Here, we choose `resolution=4` and use the one-hot embedding scheme.

In [5]:
model.qutip_setup(4, embedding_scheme='onehot', time_discretization=40, post_processing_method="TNC")
response = model.optimize(verbose=1)

Compiled.
Solved.
* Runtime breakdown
SimuQ compilation: 4.781 s
Backend runtime: 20.244 s
Decoding time: 0.000 s
Classical (Fine-tuning) time: 0.129 s
* Total time: 25.155 s

* Coarse solution
Minimizer: [1.   0.25 1.  ]
Minimum: -0.6875

* Fine-tuned solution
Minimizer: [1. 0. 1.]
Minimum: -1.0



The QuTiP solver backend returns a coarse solution at $x=[1,0.25,1]^T$, with the minimum value $-0.6875$. This is very close but not exactly the globally optimal solution at $x^*=[1,0,1]^T$. After fine-tuning, the global minimum is found.

### 4. Solve with IonQ

We can also solve the QP problem with IonQ backend. similarly, we first setup the IonQ solver, then solve with `optimize`. 

In [ ]:
model.ionq_setup(resolution=6, api_key_from_file='ionq_api_key', time_discretization=10, shots = 1000, on_simulator=True)
response = model.optimize(verbose=1)